In [1]:
# !pip install geopy

     |████████████████████████████████| 122kB 3.9MB/s eta 0:00:01


In [223]:
from geopy.geocoders import GoogleV3
geolocator = GoogleV3(api_key="")

In [5]:
from bs4 import BeautifulSoup
import requests

In [6]:
website = requests.get('https://alwaystheadventure.com/sustainability/used-gear-stores')
print(website)

<Response [200]>


In [7]:
soup = BeautifulSoup(website.content, 'html.parser')

<!DOCTYPE doctype html>

<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="initial-scale=1" name="viewport"/>
<!-- This is Squarespace. --><!-- alwaystheadventure -->
<base href=""/>
<meta charset="utf-8">
<title>A Complete Guide to Used Outdoor Gear Stores in the U.S. — Always the Adventure</title>
<link href="https://images.squarespace-cdn.com/content/v1/5790fb74b3db2bde1aa17729/1582071991473-UHBXSN0N1FJBB10YA8G1/favicon.ico?format=100w" rel="shortcut icon" type="image/x-icon"/>
<link href="https://alwaystheadventure.com/sustainability/used-gear-stores" rel="canonical"/>
<meta content="Always the Adventure" property="og:site_name">
<meta content="A Complete Guide to Used Outdoor Gear Stores in the U.S. — Always the Adventure" property="og:title"/>
<meta content="40.7207559" property="og:latitude"/>
<meta content="-74.00076130000002" prop

In [190]:
tst=soup.body \
          .find_all('article')[0] \
          .find_all('div',recursive=False)[0] \
          .find_all('div',recursive=False)[0] \
          .find_all('div',recursive=False)[0] \
          .find_all('div',recursive=False)

In [234]:
state = ''
to_store = []
for item in tst[11:]:
    if item.find('h2'):
        state = item.find('h2').text
    for name in item.find_all('h3'):
        ## check for one-off data errors
        if(name.text=='PENNSYLVANIA'):
            state = 'PENNSYLVANIA'
            continue
        if(name.text=='Before You Drive Away…'):
            break

        txt = name.text
        next_sib = name.next_sibling
        if 'Location' not in name.next_sibling.text:
            next_sib = name.next_sibling.next_sibling
        if len(next_sib.find_all('a'))==0:
            for city in next_sib.text.split(':')[1].split(','):
                    url = ''
                    if(name.a):
                        url = name.a['href']
                    to_store.append({'name': txt.strip(),
                                     'city': city.strip(),
                                     'state': state,
                                     'search_string': txt.strip()+', '+city.strip() + ', ' +state,
                                     'url':url,
                                    })
        else:
            for city in next_sib.find_all('a'):
                    to_store.append({'name': txt.strip(),
                                     'city': city.text.replace(',', '').strip(),
                                     'state': state,
                                     'search_string': txt.strip()+', '+city.text.replace(',', '').strip() + ', ' +state,
                                     'url':city['href'],
                                    })
        


In [235]:
for w in to_store:
    location = geolocator.geocode(w['search_string'])
    if location:
        w['geocode']=location[1]
        w['lat']=location[1][0]
        w['lon']=location[1][1]


In [229]:
import pandas as pd

In [236]:
pd.DataFrame(to_store).to_csv('used_gear.csv')

In [240]:
import json

In [248]:
feature_dict = [
    {
      "type": "Feature",
      "properties": {
        "_umap_options": {
          "iconClass": "Drop",
          "iconUrl": "/uploads/pictogram/shop-24_1.png",
          "color": "Yellow"
        },
        "description": "",
        "name": f"[[{i['url']}|{i['name']}]]"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          i['lon'],
          i['lat']
        ]
      }
    } for i in to_store if 'lat' in i]

with open("used_gear.geojson", "w") as outfile:
    json.dump({"type": "FeatureCollection", "features": feature_dict}, outfile, indent=4)
